<style>
    .ul{
        line-height: 250%;
    }
</style>

<font size="4">
    <ul>
        <li> calculate age by birth_time </li>
        <li> check is any people move contact_loc </li>
    </ul>
</font>


In [50]:
# basic libraries
import os
import math
import time

import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [2]:
# pyspark related
from pyspark import SparkContext
from pyspark import SparkConf

In [3]:
sc = SparkContext('local[*]', 'PySpark')

In [4]:
# spark sql related
from pyspark.sql import DataFrameWriter
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
sql = sqlContext.sql

In [5]:
sc.master

u'local[*]'

<h1> Read Profile </h1>

In [8]:
profile_path = os.path.abspath('../../dataset/profile/**/*.csv')

In [9]:
profile_txtRDD = sc.textFile(profile_path)
profile_txtRDD.take(5)

[u'1,1871111837,F,D,181903,F,373933,3320438400,1217.1769900503969,528.7309192220683,Y,Y,N,N,3696969600',
 u'2,1208855837,M,C,68383,F,72631,3688502400,1717.8806412199986,3.3682296721720717,Y,N,Y,N,3696969600',
 u'3,2880263837,M,E,457653,F,93331,3692304000,1216.7107394786008,307.4304767295353,N,N,N,Y,3696969600',
 u'4,1334999837,M,L,74257,I,248503,3647980800,1414.86660231857,888.3791238333672,Y,N,N,Y,3696969600',
 u'5,2817191837,F,A,45583,I,895863,3622406400,1388.8557936778625,642.1109125464978,Y,Y,N,Y,3696969600']

In [10]:
# profile_txtRDD.count()

100

In [11]:
profiletxtnum = _
print profiletxtnum

100


In [12]:
from pyspark.sql.types import *

In [13]:
# field of profile
profilefields = [
    StructField('customer_id', StringType(), False),
    StructField('birth_time', LongType(), True),
    StructField('gender', StringType(), True),
    StructField('contact_loc', StringType(), True),
    StructField('contact_code', StringType(), True),
    StructField('register_loc', StringType(), True),
    StructField('register_code', StringType(), True),
    StructField('start_time', LongType(), True),
    StructField('aum', DoubleType(), True),
    StructField('net_profit', DoubleType(), True),
    StructField('credit_card_flag', StringType(), True),
    StructField('loan_flag', StringType(), True),
    StructField('deposit_flag', StringType(), True),
    StructField('wealth_flag', StringType(), True),
    StructField('partition_time', LongType(), True)
]

In [14]:
# construct profile's schema
profileschema = StructType(profilefields)

In [15]:
def process_profile_line(l):
    p = l.split(',')
    # for LongType
    for i in [1, 7, 14]:
        p[i] = int(p[i])
    # for DoubleType
    for i in [8, 9]:
        p[i] = float(p[i])
    return p

In [16]:
profile_df = profile_txtRDD \
                .map(process_profile_line) \
                .toDF(schema=profileschema)

profile_df.show(5)

+-----------+----------+------+-----------+------------+------------+-------------+----------+------------------+------------------+----------------+---------+------------+-----------+--------------+
|customer_id|birth_time|gender|contact_loc|contact_code|register_loc|register_code|start_time|               aum|        net_profit|credit_card_flag|loan_flag|deposit_flag|wealth_flag|partition_time|
+-----------+----------+------+-----------+------------+------------+-------------+----------+------------------+------------------+----------------+---------+------------+-----------+--------------+
|          1|1871111837|     F|          D|      181903|           F|       373933|3320438400|1217.1769900503969| 528.7309192220683|               Y|        Y|           N|          N|    3696969600|
|          2|1208855837|     M|          C|       68383|           F|        72631|3688502400|1717.8806412199986|3.3682296721720717|               Y|        N|           Y|          N|    3696969600|


<h1> Construct The Map of Contact Location and Contact Code </h1>

In [17]:
clRDD = profile_df.select('contact_loc')

In [18]:
ccRDD = profile_df.select('contact_code')

In [19]:
clccRDD = profile_df.select('contact_loc', 'contact_code')

<h2> Calculate Numbers </h2>

In [22]:
# clnum = clRDD.distinct().count()
clnum = 11
print "total " + str(clnum) + " different contact location"

total 11 different contact location


In [23]:
# ccnum = ccRDD.distinct().count()
ccnum = 24
print "total " + str(ccnum) + " different contact code"

total 24 different contact code


In [99]:
# clccnum = clccRDD.distinct().count()
clccnum = 25
print "total " + str(clccnum) + " different pair of (location, code)"

total 25 different pair of (location, code)


<h2> Breif Survey </h2>

In [25]:
# calculate total amount of every location
cal_clRDD = clRDD.rdd \
                .map(lambda c: (c.contact_loc, 1)) \
                .reduceByKey(lambda x, y: x+y) \
                .sortByKey()

In [26]:
calcldict = cal_clRDD.collectAsMap()

In [27]:
print calcldict

{u'A': 4, u'C': 8, u'B': 6, u'E': 8, u'D': 8, u'G': 12, u'F': 4, u'H': 10, u'K': 8, u'J': 16, u'L': 16}


<h2> Check whether code is 1-1 to loc </h2>

In [89]:
check_codeTolocRDD = clccRDD.distinct().rdd \
                        .map(lambda c: (c.contact_code, [c.contact_loc])) \
                        .reduceByKey(lambda x, y: x+y) \
                        .filter(lambda (x, y): len(y) > 1)

In [90]:
check = check_codeTolocRDD.count()
# check = 1

if check == 0:
    print "1-1 checked"
else:
    print "total " + str(check) + " code have at least two location"
    print check_codeTolocRDD.take(5)

total 1 code have at least two location
[(u'457653', [u'J', u'E'])]


<h2> Construct Map </h2>

In [95]:
maplocTocodeRDD = clccRDD.distinct().rdd \
                    .map(lambda c: (c.contact_loc, [c.contact_code])) \
                    .reduceByKey(lambda x, y: x+y)

In [96]:
maplocTocodedict = maplocTocodeRDD.collectAsMap()

In [97]:
for x in maplocTocodedict.keys():
    print x
    print maplocTocodedict[x]
    print "total " + str(len(maplocTocodedict[x])) + " different code\n"

A
[u'45583']
total 1 different code

C
[u'437583', u'68383']
total 2 different code

B
[u'698061', u'115261']
total 2 different code

E
[u'457653', u'468541']
total 2 different code

D
[u'181903']
total 1 different code

G
[u'230881', u'215761', u'180201']
total 3 different code

F
[u'647221']
total 1 different code

H
[u'530713', u'159601', u'922561']
total 3 different code

K
[u'843643', u'109893']
total 2 different code

J
[u'32581', u'457653', u'48181', u'124963']
total 4 different code

L
[u'354621', u'358203', u'53593', u'74257']
total 4 different code



In [98]:
# final check
tol = 0
for x in maplocTocodedict.keys():
    tol += len(maplocTocodedict[x])

if tol == clccnum:
    print "Final check complete!"
else:
    print "Something goes wrong! Check it again"

Final check complete!


<h1> Find Partition </h1>

In [34]:
times = profile_df \
    .select('partition_time') \
    .distinct() \
    .orderBy('partition_time') \
    .rdd.map(lambda x: x.partition_time) \
    .collect()

times

[3696969600, 3699475200]

In [45]:
byPartitionDFlist = []

In [46]:
def makeDFbytime(df, time):
    timedf = df.select('*') \
                .where("partition_time = {}".format(time))
    return timedf

In [47]:
for i in xrange(len(times)):
    byPartitionDFlist.append(makeDFbytime(profile_df, times[i]))

In [48]:
byPartitionDFlist[0].first()

Row(customer_id=u'1', birth_time=1871111837, gender=u'F', contact_loc=u'D', contact_code=u'181903', register_loc=u'F', register_code=u'373933', start_time=3320438400, aum=1217.1769900503969, net_profit=528.7309192220683, credit_card_flag=u'Y', loan_flag=u'Y', deposit_flag=u'N', wealth_flag=u'N', partition_time=3696969600)

In [43]:
locToid_timedict = {}

In [55]:
def init_locToid_time(d, times, bpl):
    for i in xrange(len(times)):
        d.setdefault(times[i], {})
        bpldict = bpl[i].select('contact_loc', 'customer_id') \
                        .distinct().rdd \
                        .map(lambda li: (li.contact_loc, [li.customer_id])) \
                        .reduceByKey(lambda x, y: x+y) \
                        .collectAsMap()
        d[times[i]] = bpldict
    return d

In [56]:
starttime = time.time()

locToid_timedict = init_locToid_time(locToid_timedict, times, byPartitionDFlist)

print "total time: " + str(time.time() - starttime)

total time: 1.83449006081


In [57]:
locToid_timedict

{3696969600: {u'A': [u'5', u'17', u'42', u'30'],
  u'C': [u'39', u'2', u'14', u'27'],
  u'D': [u'26', u'13', u'1', u'38'],
  u'E': [u'3', u'28', u'40', u'15'],
  u'G': [u'45', u'20', u'33', u'8'],
  u'H': [u'31', u'21', u'6', u'46', u'25', u'9', u'50', u'18', u'43', u'34'],
  u'J': [u'47', u'24', u'10', u'22', u'37', u'35', u'49', u'12'],
  u'K': [u'11', u'23', u'36', u'48'],
  u'L': [u'19', u'29', u'41', u'16', u'44', u'32', u'4', u'7']},
 3699475200: {u'B': [u'6', u'31', u'18', u'43', u'25', u'50'],
  u'C': [u'15', u'28', u'40', u'3'],
  u'D': [u'26', u'13', u'1', u'38'],
  u'E': [u'44', u'32', u'19', u'7'],
  u'F': [u'12', u'37', u'49', u'24'],
  u'G': [u'22', u'35', u'30', u'17', u'47', u'42', u'10', u'5'],
  u'J': [u'41', u'16', u'14', u'29', u'2', u'4', u'27', u'39'],
  u'K': [u'8', u'33', u'20', u'45'],
  u'L': [u'21', u'34', u'23', u'9', u'11', u'46', u'36', u'48']}}

<h1> Read ATM </h1>

In [58]:
atm_path = os.path.abspath("../../dataset/atm/**/*.csv")

In [59]:
atm_txtRDD = sc.textFile(atm_path)
atm_txtRDD.take(5)

[u'customer_id,17,inquire,3696970704,saving_acct,MJQXLDJMQQOYAPFBBPBS,ATM,NCZHYDSTABCUAXLYJLQZ,"{\\"action\\": {\\"txn_amt\\": 0.0, \\"txn_fee_amt\\": 0.0, \\"trans_type\\": \\"trans_out\\"}, \\"object\\": {\\"target_bank_code\\": \\"B_bank\\", \\"target_acct_nbr\\": \\"UMOTEPKACILAPBIKOYHZ\\"}, \\"channel\\": {\\"address_zipcode\\": 601401, \\"machine_bank_code\\": \\"C_bank\\"}}",atm,3696969600',
 u'customer_id,42,inquire,3696970704,saving_acct,MJQXLDJMQQOYAPFBBPBS,ATM,NCZHYDSTABCUAXLYJLQZ,"{\\"action\\": {\\"txn_amt\\": 0.0, \\"txn_fee_amt\\": 0.0, \\"trans_type\\": \\"trans_out\\"}, \\"object\\": {\\"target_bank_code\\": \\"B_bank\\", \\"target_acct_nbr\\": \\"UMOTEPKACILAPBIKOYHZ\\"}, \\"channel\\": {\\"address_zipcode\\": 601401, \\"machine_bank_code\\": \\"C_bank\\"}}",atm,3696969600',
 u'customer_id,2,inquire,3696973691,saving_acct,NQVSLAHSTIYMZFTFVPBH,ATM,HFJXJQTXPAMHUNCEMXBC,"{\\"action\\": {\\"txn_amt\\": 0.0, \\"txn_fee_amt\\": 0.0, \\"trans_type\\": \\"trans_out\\"}, \\"ob

In [61]:
# atmnum = atm_txtRDD.count()
atmnum = 1934
print "total " + str(atmnum) + " atm datas"

total 1934 atm datas


In [62]:
featurefields = [
    StructField('actor_type', StringType(), False),
    StructField('actor_id', StringType(), True),
    StructField('action_type', StringType(), True),
    StructField('action_time', LongType(), True),
    StructField('object_type', StringType(), True),
    StructField('object_id', StringType(), True),
    StructField('channel_type', StringType(), True),
    StructField('channel_id', StringType(), True),
    StructField('attrs', StringType(), True),
    StructField('theme', StringType(), True),
    StructField('partition_time', LongType(), True)
]

In [63]:
featureschema = StructType(featurefields)

In [66]:
import json

def process_profile_line_feature(l):
    p = l.split(',')
    # pre
    pre_fields = p[:8]
    pre_fields[3] = int(pre_fields[3])
    # post
    post_fields = p[-2:]
    post_fields[-1] = int(post_fields[-1])
    # attrs
    attrs = json.loads(','.join(p[8:-2]))
    # concat fields
    fields = pre_fields + [attrs] + post_fields
    return fields

In [67]:
atm_df = atm_txtRDD \
    .map(process_profile_line_feature) \
    .toDF(schema=featureschema)

atm_df.show(5)

+-----------+--------+-----------+-----------+-----------+--------------------+------------+--------------------+--------------------+-----+--------------+
| actor_type|actor_id|action_type|action_time|object_type|           object_id|channel_type|          channel_id|               attrs|theme|partition_time|
+-----------+--------+-----------+-----------+-----------+--------------------+------------+--------------------+--------------------+-----+--------------+
|customer_id|      17|    inquire| 3696970704|saving_acct|MJQXLDJMQQOYAPFBBPBS|         ATM|NCZHYDSTABCUAXLYJLQZ|{"action": {"txn_...|  atm|    3696969600|
|customer_id|      42|    inquire| 3696970704|saving_acct|MJQXLDJMQQOYAPFBBPBS|         ATM|NCZHYDSTABCUAXLYJLQZ|{"action": {"txn_...|  atm|    3696969600|
|customer_id|       2|    inquire| 3696973691|saving_acct|NQVSLAHSTIYMZFTFVPBH|         ATM|HFJXJQTXPAMHUNCEMXBC|{"action": {"txn_...|  atm|    3696969600|
|customer_id|      27|    inquire| 3696973691|saving_acct|NQVSLA

In [69]:
atmtimes = atm_df \
                .select('partition_time') \
                .distinct() \
                .orderBy('partition_time') \
                .rdd.map(lambda x: x.partition_time) \
                .collect()

atmtimes

[3696969600]

In [70]:
atmbyPartitionDFlist = []

In [73]:
for i in xrange(len(times)):
    atmbyPartitionDFlist.append(makeDFbytime(atm_df, times[i]))

In [74]:
atmbyPartitionDFlist[0].first()

Row(actor_type=u'customer_id', actor_id=u'17', action_type=u'inquire', action_time=3696970704, object_type=u'saving_acct', object_id=u'MJQXLDJMQQOYAPFBBPBS', channel_type=u'ATM', channel_id=u'NCZHYDSTABCUAXLYJLQZ', attrs=u'{"action": {"txn_amt": 0.0, "txn_fee_amt": 0.0, "trans_type": "trans_out"}, "object": {"target_bank_code": "B_bank", "target_acct_nbr": "UMOTEPKACILAPBIKOYHZ"}, "channel": {"address_zipcode": 601401, "machine_bank_code": "C_bank"}}', theme=u'atm', partition_time=3696969600)

In [75]:
atmToid_timedict = {}

In [ ]:
def init_atmToid_time(d, times, bpl):
    for i in xrange(len(atmtimes)):
        d.setdefault(atmtimes[i], {})
        bpldict = bpl[i].select('actor_id', 'customer_id') \
                        .distinct().rdd \
                        .map(lambda li: (li.contact_loc, [li.customer_id])) \
                        .reduceByKey(lambda x, y: x+y) \
                        .collectAsMap()
        d[times[i]] = bpldict
    return d

<h1> Construct The Map of ATM and Address Zipcode </h1>

In [115]:
cidatRDD = atm_df.select('channel_id', 'attrs').rdd \
                        .map(lambda c: (c.channel_id, c.attrs)) \
                        .map(lambda (i, a): (i, json.loads(a)))

In [123]:
cidatRDD.take(2)

[(u'NCZHYDSTABCUAXLYJLQZ',
  {u'action': {u'trans_type': u'trans_out',
    u'txn_amt': 0.0,
    u'txn_fee_amt': 0.0},
   u'channel': {u'address_zipcode': 601401, u'machine_bank_code': u'C_bank'},
   u'object': {u'target_acct_nbr': u'UMOTEPKACILAPBIKOYHZ',
    u'target_bank_code': u'B_bank'}}),
 (u'NCZHYDSTABCUAXLYJLQZ',
  {u'action': {u'trans_type': u'trans_out',
    u'txn_amt': 0.0,
    u'txn_fee_amt': 0.0},
   u'channel': {u'address_zipcode': 601401, u'machine_bank_code': u'C_bank'},
   u'object': {u'target_acct_nbr': u'UMOTEPKACILAPBIKOYHZ',
    u'target_bank_code': u'B_bank'}})]

<h2> Check whether cid is 1-1 to Code </h2>

In [116]:
# check 1-1 of channel_id to zipcode
check_cidTocodeRDD = cidatRDD \
                        .map(lambda (i, a): (i, a['channel']['address_zipcode'])) \
                        .distinct() \
                        .map(lambda (x, y): (x, [y])) \
                        .reduceByKey(lambda x, y: x+y) \
                        .filter(lambda (x, y): len(y) > 1)

In [117]:
check = check_cidTocodeRDD.count()

if check == 0:
    print "every cid have only one code, 1-1 checked"
else:
    print "total " + str(check) + " cid have at least two code"
    print check_cidTocodeRDD.take(5)

every cid have only one code, 1-1 checked


<h2> Construct Map </h2>

In [131]:
mapcodeTocidRDD = cidatRDD \
                    .map(lambda (x, y): (y['channel']['address_zipcode'], x)) \
                    .distinct() \
                    .map(lambda (x, y): (x, [y])) \
                    .reduceByKey(lambda x, y: x+y)

In [132]:
mapcodeTociddict = mapcodeTocidRDD.collectAsMap()

In [133]:
for x in mapcodeTociddict.keys():
    print x
    print mapcodeTociddict[x]
    print "total " + str(len(mapcodeTociddict[x])) + " atm in the area\n"

782341
[u'WFNXQDBRMOJZMUTELZTC']
total 1 atm in the area

757771
[u'HNCPPYTEJXNRLLDOHMJI']
total 1 atm in the area

526351
[u'IBNNGCFIYYSEMRCGHSCL']
total 1 atm in the area

57361
[u'EGJKWYULYVHDBQJNXHFP']
total 1 atm in the area

108571
[u'XQMBMZRFWCISLSFAIFJX']
total 1 atm in the area

983073
[u'LLPKVAUIJWWIYSRPBCCI', u'RMCMNVAOUXGBTDUCOILX']
total 2 atm in the area

124963
[u'CBZLGKYZYARLNANWGODL', u'QKMQPFNJFFEDOGLTECYL']
total 2 atm in the area

436261
[u'IRFXSZEOYFMIFMDIIDET']
total 1 atm in the area

366631
[u'ARHFKQVEAGSIXWYRWCSR']
total 1 atm in the area

546861
[u'XPMPGBRFPGYMRKEDYJON']
total 1 atm in the area

41007
[u'XHZRUCDQKFNNEMZUXNRM', u'UEHLVETWLREACFNZJXXN']
total 2 atm in the area

43057
[u'IWTUCAUTJMMQXHXTEBHL']
total 1 atm in the area

647221
[u'EWXRYHPIWFBWKNACBBVM']
total 1 atm in the area

28731
[u'WKAWXIQSQMAOYLCPDWGY']
total 1 atm in the area

725053
[u'YWKWYNQQOFTZGJUULNMB']
total 1 atm in the area

497731
[u'YGIZVIPQMHIPFBZBIWIX', u'YOEVJJNLVKLMPNLZNBEH']
t

In [128]:
atmnum = atm_df.select('channel_id').distinct().count()
print "total " + str(atmnum) + " atm"

total 967 atm


In [134]:
# final check
tol = 0

for x in mapcodeTociddict.keys():
    tol += len(mapcodeTociddict[x])

if tol == atmnum:
    print "Final check complete!"
else:
    print "Something goes wrong! Check it again"

Final check complete!


<h1> Combine Location and ATM </h1>

In [130]:
# maplocTocodedict: dictionary with location as key and code list as value
# locToid_timedict: dictionary with location as key and id list as value and partition by time
# mapcodeTociddict: dictionary with code as key and channel id list as value
# codeTolocdict: dictionary with code as key and location list as value
# locTociddict: dictionary with location as key and channel id list as value
# mapcidToiddict: dictionary with channel id as key and id list as value

In [144]:
codeTolocdict = {}

for x in maplocTocodedict.keys():
    for i in xrange(len(maplocTocodedict[x])):
        cl = codeTolocdict.setdefault(maplocTocodedict[x][i], [])
        cl.append(x)

In [145]:
codeTolocdict

{u'109893': [u'K'],
 u'115261': [u'B'],
 u'124963': [u'J'],
 u'159601': [u'H'],
 u'180201': [u'G'],
 u'181903': [u'D'],
 u'215761': [u'G'],
 u'230881': [u'G'],
 u'32581': [u'J'],
 u'354621': [u'L'],
 u'358203': [u'L'],
 u'437583': [u'C'],
 u'45583': [u'A'],
 u'457653': [u'E', u'J'],
 u'468541': [u'E'],
 u'48181': [u'J'],
 u'530713': [u'H'],
 u'53593': [u'L'],
 u'647221': [u'F'],
 u'68383': [u'C'],
 u'698061': [u'B'],
 u'74257': [u'L'],
 u'843643': [u'K'],
 u'922561': [u'H']}

In [146]:
# check the code which have two location
codeTolocdict[u'457653'] = [u'J']

In [148]:
locTociddict = {}

for x in mapcodeTociddict.keys():
    if x in codeTolocdict.keys():
        lcid = locTociddict.setdefault(codeTolocdict[x][0], [])
        for i in xrange(len(mapcodeTociddict[x])):
            lcid.append(mapcodeTociddict[x][i])
    else:
        lcid = locTociddict.setdefault(u'O', [])
        for i in xrange(len(mapcodeTociddict[x])):
            lcid.append(mapcodeTociddict[x][i])

In [165]:
locTociddict

{u'O': [u'WFNXQDBRMOJZMUTELZTC',
  u'HNCPPYTEJXNRLLDOHMJI',
  u'IBNNGCFIYYSEMRCGHSCL',
  u'EGJKWYULYVHDBQJNXHFP',
  u'XQMBMZRFWCISLSFAIFJX',
  u'LLPKVAUIJWWIYSRPBCCI',
  u'RMCMNVAOUXGBTDUCOILX',
  u'CBZLGKYZYARLNANWGODL',
  u'QKMQPFNJFFEDOGLTECYL',
  u'IRFXSZEOYFMIFMDIIDET',
  u'ARHFKQVEAGSIXWYRWCSR',
  u'XPMPGBRFPGYMRKEDYJON',
  u'XHZRUCDQKFNNEMZUXNRM',
  u'UEHLVETWLREACFNZJXXN',
  u'IWTUCAUTJMMQXHXTEBHL',
  u'EWXRYHPIWFBWKNACBBVM',
  u'WKAWXIQSQMAOYLCPDWGY',
  u'YWKWYNQQOFTZGJUULNMB',
  u'YGIZVIPQMHIPFBZBIWIX',
  u'YOEVJJNLVKLMPNLZNBEH',
  u'WPMKOBRJMJFZEMRIWBTN',
  u'FAXPJAQAITMXQEFIWFOG',
  u'XAMNJUQBLPZQJGXPHXKS',
  u'JRVKMSRQAGHHTPEGOFKE',
  u'UWEQNJUXHQWAQDPCJGEX',
  u'HHIDOFRSKCOSWAOZROQK',
  u'UNBKWPIESSUQYRSEWLST',
  u'MVRVMGCBZADXPDFCGNSA',
  u'HSMJXOQQNDPMEXYMDLSE',
  u'CMOSYGRNHZLQINYSIHFX',
  u'QSUSOJOCJNERLVVKBTVC',
  u'VRZDTZPYOQVLNLIOMEFG',
  u'QNTUVKVHTKMHKOOQGNXI',
  u'VPTQYIAEUHGTUOTANBHA',
  u'LIZWAYGULDUHIAVYHCXT',
  u'KQRTKSVMWYAZRZXNSZPG',
  u'MMHGUBRZQANULMNDTN

In [155]:
mckey = mapcodeTociddict.keys()
clkey = codeTolocdict.keys()

ftol = 0
ttol = 0

for c in mckey:
    if c in clkey:
        ttol += 1
    else:
        ftol += 1
print (ttol, ftol)

(0, 588)


In [157]:
# checkpoint of ttol and ftol's 
if len(mapcodeTociddict.keys()) == (ftol + ttol):
    print "Final check complete!"
else:
    print "Something goes wrong! Check it again"

Final check complete!


In [156]:
# final check
tol = 0

for x in locTociddict.keys():
    tol += len(locTociddict[x])

if tol == atmnum:
    print "Final check complete!"
else:
    print "Something goes wrong! Check it again"

Final check complete!


<h2> Construct Relationship of channel id to channel id </h2>

In [159]:
acchRDD = atm_df.select('actor_id', 'channel_id')

In [160]:
map_calidTocidRDD = acchRDD.rdd \
                    .map(lambda i: ((i.actor_id, i.channel_id), 1)) \
                    .reduceByKey(lambda x, y: x+y)

In [171]:
map_calidTocidRDD.take(3)

[((u'26', u'PTBLYXRMUCBKRTGNMSOS'), 1),
 ((u'35', u'LLPKVAUIJWWIYSRPBCCI'), 1),
 ((u'37', u'FQOLXVXYOBYNLSRDRPHB'), 1)]

In [167]:
map_calidTocidRDD.aggregate(0, lambda x, y: x+y[1], lambda x, y: x+y)

1934

In [161]:
mapcidToidRDD = map_calidTocidRDD \
                    .map(lambda (x, y): x) \
                    .map(lambda (x, y): (y, x)) \
                    .distinct() \
                    .map(lambda (x, y): (x, [y])) \
                    .reduceByKey(lambda x, y: x+y)

In [163]:
mapcidToiddict = mapcidToidRDD.collectAsMap()

In [164]:
mapcidToiddict

{u'AAMLOHGKAUKRMTUXNVSN': [u'10', u'35'],
 u'ACFJQSNCUYYZLZFYUSNJ': [u'34', u'9'],
 u'ADACKZDXELXIYPQLZTRY': [u'41', u'16'],
 u'ADKQSIFONBVHWVITEUFB': [u'25', u'50'],
 u'AEPDOJXICJDGKLCMAHEM': [u'8', u'33'],
 u'AEREVXPSHSTZBHNWWFEU': [u'9', u'34'],
 u'AEYYKDOOUDIEHPLCXFEH': [u'6', u'31'],
 u'AFUOHAXAWWEXZNEBMBXJ': [u'24', u'49'],
 u'AGVMPNCQYVTOFSTPKLYU': [u'34', u'9'],
 u'AHFASRNJUUGYGJCYWXFK': [u'15', u'40'],
 u'AIPNAMTKCSGIXYNHCATX': [u'9', u'34'],
 u'ALCOHFHSLQSPBGXJRHYX': [u'48', u'23'],
 u'ALCVRTZMVLDRHFPMPRHM': [u'45', u'20'],
 u'ALYKDLNYJQCLUKVMMCVH': [u'26', u'1'],
 u'AMHZICDHNKQAZTDTZRNG': [u'20', u'45'],
 u'AMPJZARQJFCBNISQXVGL': [u'10', u'35'],
 u'ANKYYKHFCMNQCCRBEUVE': [u'13', u'38'],
 u'AOAZMKNSQKHTISIJMHNP': [u'5', u'30'],
 u'APIOUSKHTDNKQAJRJQIW': [u'50', u'25'],
 u'APKYJFBFGTZWKXWZNGZZ': [u'46', u'21'],
 u'APMTAFGHJFWMPLQEGCPK': [u'48', u'23'],
 u'APQLMPLXBHPAOVYDPEML': [u'23', u'48'],
 u'APZYKXJDKBWTRXEJHPCT': [u'31', u'6'],
 u'ARHFKQVEAGSIXWYRWCSR': [u'38', u'13'],
 

In [168]:
mapidTociddict = {}

for x in mapcidToiddict.keys():
    for i in xrange(len(mapcidToiddict[x])):
        cidd = mapidTociddict.setdefault(mapcidToiddict[x][i], [])
        cidd.append(x)

In [169]:
mapidTociddict

{u'1': [u'OHRDIOEPBMSMEYUZQCXZ',
  u'UEUIIXVVBKKIRHRRSKKM',
  u'YIIUGTOACFLWIIJJPWAH',
  u'ODAJAGWYTMYCFACBGOHJ',
  u'PTBLYXRMUCBKRTGNMSOS',
  u'WXZHTNBSYJBHVGDKHOPH',
  u'BFUGQMLTQBIBLDPUZEQW',
  u'KLAXLRDTPRUJVFJAOIDT',
  u'NTOILSVSHJQAMDRWXQMM',
  u'ALYKDLNYJQCLUKVMMCVH',
  u'RZKFNNFWHOMJUTVIMPPI',
  u'GYJYEASIBZKLEXERFWWB',
  u'YPAUKKNYJLDHQIWFBAPM',
  u'GNDUXYSGYKXPHCYEEGIQ',
  u'RHIGXFHSYJYDMMRKEZGT',
  u'HAPKLJNLATOCYPGNFXWB',
  u'IYBGWHIPWNIRERUAJQYW',
  u'MEFZKKRFKODKBLLPJTHX',
  u'FZKXEUINCRSADPQQHOBX',
  u'QFOFVMOFHBHWUDYBHUOB',
  u'RFGNBGJEWJBHESAMBMWS',
  u'JIJTDPTZNSWWYLHLZJLM',
  u'RSTZZBKGGDYYCDBXCRJB',
  u'LZBIJPLNGRTWMNXCXCHT',
  u'VWHVPKGCBGBNKRYKWQYD',
  u'UNBKWPIESSUQYRSEWLST',
  u'GXMQWZLURRMTJALQBAYE',
  u'VBBMYOFEOVFJXFBGGEAS',
  u'XLYYHJBXEFUBSRZOFRXQ',
  u'CDJUVJYDPAYIINKUUPSM',
  u'LTEVYCDDVVUYGIXVKFOY',
  u'LRLFGFGMAOLVPUTVPISK',
  u'XNASHVNSQCNUGRRHTQQK',
  u'ZFPREHQIQBPTWRBBNPOI',
  u'QTOXOOVWQBEUSWTUPDBR',
  u'WVKQCVVHUTRHKTNVKFOH',
  u'NURQCKNIHJWQYXWRAQ

<h1> Produce The Dot Code </h1>

In [173]:
# convert to UTF-8
mapidTociddictencode = {}

for x in mapidTociddict.keys():
    for i in xrange(len(mapidTociddict[x])):
        encidd = mapidTociddictencode.setdefault(x.encode("UTF-8"), [])
        encidd.append(mapidTociddict[x][i].encode("UTF-8"))

In [174]:
mapidTociddictencode

{'1': ['OHRDIOEPBMSMEYUZQCXZ',
  'UEUIIXVVBKKIRHRRSKKM',
  'YIIUGTOACFLWIIJJPWAH',
  'ODAJAGWYTMYCFACBGOHJ',
  'PTBLYXRMUCBKRTGNMSOS',
  'WXZHTNBSYJBHVGDKHOPH',
  'BFUGQMLTQBIBLDPUZEQW',
  'KLAXLRDTPRUJVFJAOIDT',
  'NTOILSVSHJQAMDRWXQMM',
  'ALYKDLNYJQCLUKVMMCVH',
  'RZKFNNFWHOMJUTVIMPPI',
  'GYJYEASIBZKLEXERFWWB',
  'YPAUKKNYJLDHQIWFBAPM',
  'GNDUXYSGYKXPHCYEEGIQ',
  'RHIGXFHSYJYDMMRKEZGT',
  'HAPKLJNLATOCYPGNFXWB',
  'IYBGWHIPWNIRERUAJQYW',
  'MEFZKKRFKODKBLLPJTHX',
  'FZKXEUINCRSADPQQHOBX',
  'QFOFVMOFHBHWUDYBHUOB',
  'RFGNBGJEWJBHESAMBMWS',
  'JIJTDPTZNSWWYLHLZJLM',
  'RSTZZBKGGDYYCDBXCRJB',
  'LZBIJPLNGRTWMNXCXCHT',
  'VWHVPKGCBGBNKRYKWQYD',
  'UNBKWPIESSUQYRSEWLST',
  'GXMQWZLURRMTJALQBAYE',
  'VBBMYOFEOVFJXFBGGEAS',
  'XLYYHJBXEFUBSRZOFRXQ',
  'CDJUVJYDPAYIINKUUPSM',
  'LTEVYCDDVVUYGIXVKFOY',
  'LRLFGFGMAOLVPUTVPISK',
  'XNASHVNSQCNUGRRHTQQK',
  'ZFPREHQIQBPTWRBBNPOI',
  'QTOXOOVWQBEUSWTUPDBR',
  'WVKQCVVHUTRHKTNVKFOH',
  'NURQCKNIHJWQYXWRAQSM',
  'UGIQBMRFKZQNSYTWKRWB',
  'RENJ

In [176]:
for x in mapidTociddictencode.keys():
    for i in xrange(len(mapidTociddictencode[x])):
        print x + " -> " + mapidTociddictencode[x][i]

30 -> BEYIDHYVYRWMQKUYDKTF
30 -> NEFVQJBYNJOUKDCELXWX
30 -> TNHGTQVBWEBBHTNRJEUL
30 -> VJAVPXWXGJDSGOSBKYAW
30 -> ITFYAPPHBCPRNBKNKMER
30 -> QENCSXOKMCLVICGYTPOO
30 -> YTNMPLTVJUBANGYKYQPD
30 -> RFNOMSYJBFQXRMMBZMNA
30 -> IARJBSQDHXJIBSNNLMOR
30 -> DDLBUXWPXMFKFZXGTCWU
30 -> PZSZFZYTPHHKPSRQBVBS
30 -> KXCXOTRCBDLDTIXRJQDZ
30 -> ETIWCMJRNMWJKSMSNSZH
30 -> NNKGEQZDSGNQWTIRONKJ
30 -> ZZXDHBLHCXVULVPUKWKD
30 -> SIYCYXNPMGECLPMXCSMU
30 -> VRJDKKIRINAQPLIHAIZJ
30 -> BAVORKLBTCTKUEHWWYDX
30 -> XCOECRDHLRLGQLLVQYRH
30 -> ZZFHRNBHBQQLCLWFFFXV
30 -> ZFVIWAMXJNUYXBGSEVMS
30 -> MWUSCLISGSYIFFQHIQAY
30 -> HAEOMVVXHXNGRLVDOROK
30 -> VZSQXXIGNCQNYJUXOBVS
30 -> AOAZMKNSQKHTISIJMHNP
30 -> TCHHGCMARQGSJAJBWBSG
30 -> WNZMDHFMWVPYFFBINJLQ
30 -> NWCGBOTDMOOMAXPPWAQW
30 -> EJCZFJSLGFSFFWFFIXAC
30 -> MWRLOPZBNORFZAXJIPEL
30 -> RIAARXMPIWJTZPYXUZUZ
30 -> LJKTNOFDJGQOUIMOLBNO
30 -> EQUGRCNCVTTDMDPEKNKT
30 -> YHHTXRIWOZUQGYJURIYN
30 -> EVYRWLTMZWDVINZGSAVZ
30 -> WWMKOHZCLXCNWBHPGYPZ
30 -> MQGLJVTHKABZKSZMVOKQ
3